In [1]:
#!/usr/bin/python

import time
import numpy as np
from scipy.optimize import fsolve
from scipy.constants import c, pi, G
import scipy.integrate as integrate
import scipy.interpolate as scipy_interpolate
import astropy.units as u
import random

from numba import jit, njit

#For measuring execution time 
begin = time.time()

#Global constants  
c *= 1e-3 #convert speed of light from m/s to km/s
G *= 1e-9*5.028e31 #convert G from m^3 kg^-1 s^-2 to km^3 M_sun^-1 s^-2

In [2]:
### Chirp mass

@njit
def obs_chirp_mass(m1,m2,z):
    M_c = (1.+z) * (m1*m2)**(3./5) / (m1+m2)**(1./5)
    return M_c

### Observable mass 
@njit
def obs_total_mass(m1,m2,z):
    M_obs = (1.+z) * (m1 + m2)
    return M_obs

### Random masses m1 and m2
def get_m1s(sd, Nsize):
    np.random.seed(sd)
    m1s_arr = np.random.uniform(low=1, high=2.5, size=Nsize)
    return m1s_arr

def get_m2s(sd, Nsize):
    np.random.seed(sd)
    m2s_arr = np.random.uniform(low=1, high=2.5, size=Nsize)
    return m2s_arr


In [3]:
### sky localization angles

def get_thetas(sd, Nsize):
    np.random.seed(sd)
    thetas_arr = np.random.uniform(low=0, high=pi, size=Nsize)
    return thetas_arr 
   
def get_phis(sd, Nsize):
    np.random.seed(sd)
    phis_arr = np.random.uniform(low=0, high=2*pi, size=Nsize)
    return phis_arr

### polarization angle    
def get_psis(sd, Nsize):
    np.random.seed(sd)
    psis_arr = np.random.uniform(low=0, high=2*pi, size=Nsize)
    return psis_arr

### inclination angle 
def get_iotas(sd, Nsize):
    np.random.seed(sd)
    iotas_arr = np.random.uniform(low=0, high=pi, size=Nsize)
    return iotas_arr


In [4]:
# Antenna pattern functions with angles:(theta, phi)[sky localization], psi(polarization) and iota(inclination)  
# Following arXiv:2205.11221

#For L-shaped interferometers (LIGO, VIRGO, KAGRA and CE), the antenna pattern functions are (eq.19):
@njit
def Fp_i_square(th, ph, ps, k=1./2):
    t1 = (1.+np.cos(th)**2) * np.cos(2*ph) * np.cos(2*ps)
    t2 = 2 * np.cos(th) * np.sin(2*ph) * np.sin(2*ps) 
    return k * (t1 - t2)

@njit
def Fx_i_square(th, ph, ps, k=1./2):
    t1 = (1.+np.cos(th)**2) * np.cos(2*ph) * np.sin(2*ps)
    t2 = 2 * np.cos(th) * np.sin(2*ph) * np.cos(2*ps) 
    return k * (t1 + t2)

#For the triangle-shaped ET antenna, the functions include a extra factor srtq(3)/2
#and the phi angles are replaced by phi + 2pi/3 and phi + 4pi/3
#Then, we can call as:
# Fp_i_square(k=np.sqrt(3)/4., th, ph+(2*pi/3.), ps)
# Fx_i_square(k=np.sqrt(3)/4., th, ph+(4*pi/3.), ps)

#L-shaped (eq.23)
@njit
def F_i_squareL(th, ph, ps, io, k=1./2): #theta, phi, psi, iota
    fct = ((1.+np.cos(io)**2)/2.)**2
    return fct * Fp_i_square(th, ph, ps, k)**2 + np.cos(io)**2 * Fx_i_square(th, ph, ps, k)**2

#Triangle-shaped (eq.23)
@njit
def F_i_squareT(th, ph, ps, io, k=np.sqrt(3)/4.): #theta, phi, psi, iota
    fct = ((1.+np.cos(io)**2)/2.)**2
    return fct * Fp_i_square(th, ph, ps, k)**2 + np.cos(io)**2 * Fx_i_square(th, ph, ps, k)**2

# Functions and integrals to get the SNR. Following arXiv:2205.11221

### One-sided noise power spectral density eq.18
@njit
def Sh(f,sel_det): 
    if sel_det=='ET':
        sh = np.interp(f, f_ET, Sh_ET) #Einstein Telescope 
    elif sel_det=='CE':
        sh = np.interp(f, f_CE, Sh_CE) #Cosmic Explorer
    elif sel_det=='CES':
        sh = np.interp(f, f_CES, Sh_CES) #Cosmis Explorer South
    return sh

### Integral term of eq.22 
@njit
def freq_integrand(f,sel_det):
    return f**(-7/3.) / Sh(f,sel_det)**2

### Integration limits 
@njit
def f_upper(m1, m2, z):
    f_up = c**3 / (6*np.sqrt(6)* pi * obs_total_mass(m1,m2,z) * G)
    return f_up

### The SNR computation (eq.22) for triangle-shaped 
def SNR_rho2T(z, dl, m1, m2, th, ph, ps, io, sel_det):
    #luminosity distance must be in km
    fct1 = 5 * (G*obs_chirp_mass(m1,m2,z))**(5/3) * F_i_squareT(th, ph, ps, io, k=np.sqrt(3)/4.)
    fct2 = 6 * c**3 * pi**(4/3) * dl**2
    f_lo = 1
    f_up = f_upper(m1, m2, z)
    f_int = integrate.quad(freq_integrand, f_lo, f_up, args=(sel_det,))[0]
    return (fct1/fct2) * f_int

### The SNR computation L-shaped 
def SNR_rho2L(z, dl, m1, m2, th, ph, ps, io, sel_det):
    #luminosity distance must be in km
    fct1 = 5 * (G*obs_chirp_mass(m1,m2,z))**(5/3) * F_i_squareL(th, ph, ps, io, k=1/2.)
    fct2 = 6 * c**3 * pi**(4/3) * dl**2
    f_lo = 1
    f_up = f_upper(m1, m2, z)
    f_int = integrate.quad(freq_integrand, f_lo, f_up, args=(sel_det,))[0]
    return (fct1/fct2) * f_int


In [5]:
#Loading the sensitivities 
path = '/Users/mariamac/Dropbox/projects2021/MockGW_plots/' 

### Get the ET sensitivities 

# For more details in ET sensitivities, see: https://www.et-gw.eu/index.php/etsensitivities
# File downloaded: https://apps.et-gw.eu/tds/?content=3&r=14065
ETfile = 'gw_sensitivity_curves/ET-0000A-18_ETDSensitivityCurveTxtFile.txt' 
ET_sensitivity = np.genfromtxt(path+ETfile, names=['freq', 'et_dlf', 'et_dhf','et_dsum']) #freq in Hz

### Get the CE sensitivities 

# CE sensitivities from https://dcc.cosmicexplorer.org/cgi-bin/DocDB/ShowDocument?docid=T2000017
# See https://arxiv.org/abs/2201.10668 or https://arxiv.org/abs/2109.09882
# The baseline 40 km detector
CE40file = 'gw_sensitivity_curves/ce_strain/cosmic_explorer_strain.txt'
CE40_sensitivity = np.genfromtxt(path+CE40file, names=['freq', 'ce_40km']) 

# The baseline 20 km detector ("compact binary tuned")
# Cosmic Explorer South is assumed to be a 20 km postmerger optimized detector
# See section 2.1 and 3.1 of https://arxiv.org/pdf/2201.10668.pdf
CE20file = 'gw_sensitivity_curves/ce_strain/cosmic_explorer_20km_pm_strain.txt'
CE20_sensitivity = np.genfromtxt(path+CE20file, names=['freq', 'ce_20km']) 


In [6]:
#renaming frequency (f) and sensitivity (S_h) for ET
f_ET, Sh_ET = ET_sensitivity['freq'], ET_sensitivity['et_dsum']

#For CE
f_CE, Sh_CE = CE40_sensitivity['freq'], CE40_sensitivity['ce_40km']

#For CES
f_CES, Sh_CES = CE20_sensitivity['freq'], CE20_sensitivity['ce_20km']


In [7]:
%%time

strfile = 'dL_lcdm_md14_inv_uni_1yr.txt' #DEmodel = 'LCDM'

data = np.genfromtxt(path+strfile) 

#redshifts and luminosity distance 
z  = data[:,0]
dL = data[:,1] #in Gpc

### take a subsample for testing
idx=random.sample(range(len(z)),1000)
z = z[idx]
dL = dL[idx]
dL_km = dL*u.Gpc.to(u.km) #dL in km

#sample size
N=len(z)
print('Sample size = ', N)

#random masses
m1s = get_m1s(10, N)
m2s = get_m2s(11, N)

#random angles
thetas = get_thetas(12, N)
phis   = get_phis(13, N)
psis   = get_psis(14, N)
iotas  = get_iotas(15, N)

#get integration limits
#f_upp = f_upper(m1s, m2s, z) 
#f_low = np.ones(len(f_upp))

Sample size =  1000
CPU times: user 1.63 s, sys: 138 ms, total: 1.77 s
Wall time: 1.85 s


In [8]:
%%time

#Compute SNR ET
SNRs_rho2_ET = []
snr1, snr2, snr3 = [],[],[]
for i in range(len(z)):
    snr1 = SNR_rho2T(z[i], dL_km[i], m1s[i], m2s[i], thetas[i], phis[i], psis[i], iotas[i], 'ET')
    snr2 = SNR_rho2T(z[i], dL_km[i], m1s[i], m2s[i], thetas[i], phis[i]+(2*pi/3.), psis[i], iotas[i], 'ET')
    snr3 = SNR_rho2T(z[i], dL_km[i], m1s[i], m2s[i], thetas[i], phis[i]+(4*pi/3.), psis[i], iotas[i], 'ET')
    SNRs_rho2_ET.append(snr1+snr2+snr3) #SNRs_ET = np.sqrt(SNRs_rho2_ET)

SNRs_rho2_ET = np.array(SNRs_rho2_ET)    

/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:66: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:66: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.


CPU times: user 16 s, sys: 89.7 ms, total: 16.1 s
Wall time: 16.2 s


In [9]:
%%time

#Compute SNR CE
SNRs_rho2_CE = []
snr1 = []
for i in range(len(z)):
    snr1 = SNR_rho2L(z[i], dL_km[i], m1s[i], m2s[i], thetas[i], phis[i], psis[i], iotas[i], 'CE')
    SNRs_rho2_CE.append(snr1) # SNRs_CE = np.sqrt(SNRs_rho2_CE)

SNRs_rho2_CE = np.array(SNRs_rho2_CE)

/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


CPU times: user 5.27 s, sys: 35.9 ms, total: 5.3 s
Wall time: 5.31 s


In [10]:
%%time

#Compute SNR CES
SNRs_rho2_CES = []
snr1 = []
for i in range(len(z)):
    snr1 = SNR_rho2L(z[i], dL_km[i], m1s[i], m2s[i], thetas[i], phis[i], psis[i], iotas[i], 'CES')
    SNRs_rho2_CES.append(snr1) # SNRs_CES = np.sqrt(SNRs_rho2_CES) 

SNRs_rho2_CES = np.array(SNRs_rho2_CES)    

/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
/Users/mariamac/miniconda3/envs/mariaenv/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


CPU times: user 5.09 s, sys: 16.9 ms, total: 5.11 s
Wall time: 5.11 s


In [11]:
%%time 
#The final SNRs 

ETsnr  = np.sqrt(SNRs_rho2_ET)
CEsnr  = np.sqrt(SNRs_rho2_CE)
CESsnr = np.sqrt(SNRs_rho2_CES)
Allsnr = np.sqrt(SNRs_rho2_ET+SNRs_rho2_CE+SNRs_rho2_CES)


CPU times: user 197 µs, sys: 192 µs, total: 389 µs
Wall time: 308 µs


In [12]:
import time
#Printing running time
end = time.time()
print('Total time: ', (end - begin)/60., 'min')


Total time:  0.47612774769465127 min
